In [8]:
!rm -rf ./experiments
%config InlineBackend.figure_format = 'retina'

# Usage

<div align="center">
    <img src="logo.svg" style="width: min(100%, 450px)">
</div>

`digital-experiments` works straight out of the box:

In [1]:
from digital_experiments import experiment

@experiment
def square(x):
    return x * x

[square(i) for i in range(5)]

[0, 1, 4, 9, 16]

## Observations

In [2]:
square.observations()

[Observation(2023-12-06_16-09-59_296146, {'x': 0} → 0),
 Observation(2023-12-06_16-09-59_296393, {'x': 1} → 1),
 Observation(2023-12-06_16-09-59_296550, {'x': 2} → 4),
 Observation(2023-12-06_16-09-59_296678, {'x': 3} → 9),
 Observation(2023-12-06_16-09-59_296801, {'x': 4} → 16)]

Each [Observation](core-api.rst#digital_experiments.core.Observation) object is a light wrapper around:

- a unique identifier (implemented as a timestamped string)
- the exact configuration (args, kwargs and defaults) used to run the experiment
- the result of the experiment (the return value of the function)
- a dictionary of metadata that internal and user-defined [callback hooks](callbacks-api.rst) can use to store other relevant information

In [3]:
import json

_dict = square.observations()[0]._asdict()
print(json.dumps(_dict, indent=4))

{
    "id": "2023-12-06_16-09-59_296146",
    "config": {
        "x": 0
    },
    "result": 0,
    "metadata": {
        "timing": {
            "total": {
                "start": "2023-12-06 16:09:59",
                "end": "2023-12-06 16:09:59",
                "duration": 4e-06
            }
        },
        "code": "@experiment\ndef square(x):\n    return x * x\n"
    }
}


By default, `digital-experiments` times how long the experiment took to run, and the exact code that was run. The latter is particularly useful when we're rapdily iterating on an experiment's code, and want to be able to reproduce the results of a previous run. Extra timing information can be added to this metadata by using the [time_block](core-api.rst#digital_experiments.time_block) context.

## Backends

By default, `digital-experiments` stores each observation in its own `.pkl` file located at `./experiments/<experiment_name>/<id>.pkl`:

In [4]:
!ls ./experiments/square

2023-12-06_16-09-59_296146.pkl 2023-12-06_16-09-59_296678.pkl
2023-12-06_16-09-59_296393.pkl 2023-12-06_16-09-59_296801.pkl
2023-12-06_16-09-59_296550.pkl


Other backends are available (see the [complete list here](backends-api.rst)), or you can [implement your own](backends-api.rst).

You can also specify the root directory for a given experiment by passing the `root` argument to the [@experiment](core-api.rst#digital_experiments.experiment) decorator, or by setting the `DE_ROOT` environment variable:

In [5]:
from pathlib import Path

@experiment(root=Path("some/other/path"))
def cube(x):
    return x * x * x

## Artefacts

`digital-experiments` assigns and provides a unique directory on disk per run of an experiment. This can be accessed within an experiment using the [current_dir](core-api.rst#digital_experiments.current_dir) function. Any files saved to this directory during the experiment are available _post hoc_ via the [artefacts](core-api.rst#digital_experiments.core.Experiment.artefacts) function.

In [6]:
from digital_experiments import current_dir

@experiment
def saving_experiment():
    (current_dir() / 'test.txt').write_text('hello world')

saving_experiment()
id = saving_experiment.observations()[0].id
saving_experiment.artefacts(id)

[PosixPath('experiments/saving_experiment/storage/2023-12-06_16-09-59_448265/test.txt')]